In [2]:
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import os
%run ../functions.ipynb

In [3]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return edge, blur

In [4]:
def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## 測試擴增Edge 試圖建立虛擬連結

### Version 1 移動有問題的點

In [5]:
def move_edge(edited_edge, edge, img):
    
    edited_index = []
    for index, point in enumerate(edge):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_edge.append([x+j, y+i])
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)
    return edited_edge

### Versoin 2 直接增加可能的點

In [16]:
def add_edge(edited_edge, edge_points, img):
    
    for index, point in enumerate(edge):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_edge.append([x+j, y+i])
    
    return edited_edge

### Version 3 直接去除問題點

In [7]:
def del_edge(edited_edge, edge_points, img):
    
    edited_index = []
    for index, point in enumerate(edge):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)
    return edited_edge

## 去除上下左右極端點

In [ ]:
def remove_extreme(edited_edge, img):
    
    
    return edited_edge

In [26]:
a = np.array([[2,'a'],
              [1,'c'],
              [3,'b']])
b = np.sort(a,axis=-1)
b

array([['2', 'a'],
       ['1', 'c'],
       ['3', 'b']], dtype='<U21')

## 編輯邊緣

In [8]:
def edit_edge(edge, img, funct):
    edited_edge = edge.copy()
    
    if funct == 0:
        edited_edge = move_edge(edited_edge, edge, img)
    elif funct == 1: 
        edited_edge = add_edge(edited_edge, edge, img)
    else:
        edited_edge = del_edge(edited_edge, edge, img)
    
    edited_edge = np.unique(edited_edge, axis=0)
    return edited_edge

In [17]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
img_edge, img_blur = image_processing(img)
edge = get_edge_points(img_edge)

edited_edge = edit_edge(edge, img_blur, funct=1)

cx,cy,r,_ = cf.least_squares_circle((edge))
print("{}  {}".format(cx, cy))
cx,cy,r,_ = cf.least_squares_circle((edited_edge))
print("{}  {}".format(cx, cy))

203.43340624689515  99.48214409869645
203.39608778343222  99.51262931804382


In [18]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
try:
    
    images = os.listdir()
    images.sort()
    
    predictions = []
    for image in images:
        img = cv2.imread(image,0)
        img_edge, img_blur = image_processing(img)
        edge = get_edge_points(img_edge)
        edited_edge = edit_edge(edge, img_blur, funct=1)

        cx,cy,r,_ = cf.least_squares_circle((edited_edge))
        predictions.append([cx, cy])
        print("{}: {}, {}".format(image ,cx, cy))
        
except Exception as e:
    print(e)

10_mean_0.bmp: 203.65148127383424, 99.34046385032879
10_mean_1.bmp: 203.7226536236001, 99.20579779717248
10_mean_10.bmp: 203.72823483247674, 99.01548571141475
10_mean_11.bmp: 203.61724093797685, 99.19294420801616
10_mean_12.bmp: 203.58303395297943, 99.14008903331438
10_mean_13.bmp: 203.55023764240585, 99.16971108365122
10_mean_14.bmp: 203.50887671972993, 99.26349689932998
10_mean_15.bmp: 203.48595752271643, 99.1558225581777
10_mean_16.bmp: 203.5048771327654, 99.17142677078748
10_mean_17.bmp: 203.64995591694097, 99.29816654594798
10_mean_18.bmp: 203.42189430103605, 99.05786894482877
10_mean_19.bmp: 203.656517896485, 99.12014932238354
10_mean_2.bmp: 203.46014946699106, 98.93144635525327
10_mean_20.bmp: 203.48511322568797, 99.11745813105705
10_mean_21.bmp: 203.5854814148023, 99.2718272834331
10_mean_22.bmp: 203.5460648651913, 99.16104132544415
10_mean_23.bmp: 203.5940910388126, 99.11659210704808
10_mean_24.bmp: 203.77382769146593, 99.0433502550144
10_mean_25.bmp: 203.67622274839422, 99.21

In [19]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (203.61311, 99.17669)
Variance     : (0.01226, 0.01900)
Max Length   : 0.47077, 0.61242 (pixels)


In [20]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file_1 = "10_mean_63.bmp"
img_1 = cv2.imread(file_1,0)
blur_1 = cv2.GaussianBlur(img_1, (5,5), 0)
nlm_1 = cv2.fastNlMeansDenoising(blur_1, None, 10, 7, 21)
_, otsu_1 = cv2.threshold(nlm_1, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
edge_1 = cv2.Canny(otsu_1, 0, 20)
edge_points_1 = get_edge_points(edge_1)


# file_2 = "10_mean_81.bmp"
# img_2 = cv2.imread(file_2, 0)
# blur_2 = cv2.GaussianBlur(img_2, (5,5), 0)
# nlm_2 = cv2.fastNlMeansDenoising(blur_2, None, 10, 7, 21)
# _, otsu_2 = cv2.threshold(nlm_2, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# edge_2 = cv2.Canny(otsu_2, 0, 20)# edge_points_2 = get_edge_points(edge_2)


# 顯示圖片
cv2.imshow("Blur 1", blur_1)
# cv2.imshow("Blur 2", blur_2)

cv2.imshow("NLM 1", nlm_1)
# cv2.imshow("NLM 2", nlm_2)

cv2.imshow("Otsu 1", otsu_1)
# cv2.imshow("Otsu 2", otsu_2)

cv2.imshow("Edge 1", edge_1)
# cv2.imshow("Edge 2", edge_2)
cv2.waitKey(0)
cv2.destroyAllWindows()

{(1, 2), (3, 4), (5, 6)}

## 測試多次隨機取子點擬合的分佈
假定平均 cx: 203.61381  cy: 99.17248
### 結論：無法單靠尋找 Fram 各自的 Median 或 多次 Random subsampled 做連結

In [50]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
edge = image_processing(img)
edge_points = get_edge_points(edge)


xs = []
ys = []
for iter in range(10):
    sampled_indices = np.random.choice(range(len(edge_points)), size=int(len(edge_points)), replace=False)
    sample_points = edge_points[sampled_indices]
#     cx,cy,r,_ = cf.least_squares_circle((sample_points))
    cx = np.median(sample_points[:,0])
    cy = np.median(sample_points[:,1])
    xs.append(cx)
    ys.append(cy)

print(xs)
print(ys)

[205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0]
[100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5]
